# 인공지능공학융합특론 05월17일 과제

### 과제
- Max(Min)-Cut 알고리즘의 이해
- 주제 자유 선정 ( 회식자리에서 부장,과장,대리,사원,인턴 친밀도에 의한 자리배치 )
- 친밀도 (부장-과장 8, 부장-대리 3, 부장-사원 1, 부장-인턴 1, 과장-대리 7, 과장-사원 2, 대리-사원 6, 대리-인턴 4, 사원-인턴 5)
- Goal : 서로 친한 사람들이 마주보게 자리 배치를 하는것
- 원형테이블을 반으로 나누듯 두 그룹으로 나누고 서로 다른 그룹에 앉은 사람들 간의 친밀도 총합이 최대가 되도록 하는것
- 파일은 총 2개 ( 일반CPU로 작성할때와 Qiskit 으로 작성할때 )

### 실행
- 일반 CPU로 돌릴때 몇개의 노드까지 처리가 되는지 확인
- qiskit 을 이용한 코드작성 후 ibm quantum 비교



In [ ]:
import time
import random
from collections import defaultdict

# 친밀도 데이터 정의
closeness = {
    ('부장', '과장'): 8,
    ('부장', '대리'): 3,
    ('부장', '사원'): 1,
    ('부장', '인턴'): 1,
    ('과장', '대리'): 7,
    ('과장', '사원'): 2,
    ('대리', '사원'): 6,
    ('대리', '인턴'): 4,
    ('사원', '인턴'): 5
}

In [ ]:
def generate_set(nodes) :
  """Dict 기본세팅 """
  graph = defaultdict(dict)
  positions = [f'직원{i}' for i in range(nodes)]

  f_positions = ['부장','과장','대리','사원','인턴']
  f_graph = defaultdict(dict)

  # 기초세팅
  for (p1, p2), weight in closeness.items() :
    f_graph[p1][p2] = weight
    f_graph[p2][p1] = weight

  # 5명 데이터를 복사
  for i in range(min(5, nodes)) :
    for j in range(i+1, min(5, nodes)) :
      p1, p2 = f_positions[i], f_positions[j]
      if p2 in f_graph[p1] :
        graph[positions[i]][positions[j]] = f_graph[p1][p2]
        graph[positions[j]][positions[i]] = f_graph[p1][p2]

  # 추가 노드에 대한 랜덤 친밀도 생성
  for i in range(5, nodes) :
    for j in range(i+1, nodes) :
      weight = random.randint(1, 10)
      graph[positions[i]][positions[j]] = weight
      graph[positions[j]][positions[i]] = weight

  return graph, positions

In [ ]:
def node_performance() :
  """ 성능 테스트 실행 """
  node_counts = list(range(2, 21))
  execution_times = []
  max_nodes = 0

  print("\n=== Max-Cut 알고리즘 성능 테스트 ===")

  for n in node_counts :
    graph, positions = generate_set(n)

    start_time = time.time()
    try:
      max_cut, partition = max_cut_algorithm(graph, positions)
      execution_time = time.time() - start_time
      execution_times.append(execution_time)

      print(f"\n노드 : {n}, 실행시간 : {execution_time : .2f}, 가중치합 : {max_cut}")

      if execution_time < 60 :
        max_nodes = n
      else :
        print("\n실행 시간이 60초를 초과하여 테스트가 중단되었습니다.")
        break
    except Exception as e :
      print(f"\n노드 : {n}, 오류발생 : {str(e)}")
      break
  print("\n=== 테스트 결과 요약 ===")
  print(f"\n최대 처리 가능 노드 수 (60초 이내): {max_nodes}")
  print("\n노드 수별 실행 시간:")

  for i, n in enumerate(node_counts[:len(execution_times)]):
      print(f"{n} 노드 : {execution_times[i]:.2f} 초")


In [ ]:
def max_cut_algorithm(graph, positions) :
  """알고리즘 계산 """
  n = len(positions)
  max_cut = 0
  f_partition = None

  # 모든 가능한 분할 시도 (2^(n-1)-1)
  for i in range(1, 2**(n-1)) :
    partition = []
    for j in range(n) :
      if(i >> j) & 1 :
        partition.append(positions[j])

    # cut 크기 계산
    cut_size = 0
    for u in partition :
      for v in positions :
        if v not in partition and v in graph[u] :
          cut_size += graph[u][v]

    if cut_size > max_cut :
      max_cut = cut_size
      f_partition = (partition, [v for v in positions if v not in partition ])

  return max_cut, f_partition

In [ ]:
if __name__ == "__main__" :
  node_performance()


=== Max-Cut 알고리즘 성능 테스트 ===

노드 : 2, 실행시간 :  0.00, 가중치합 : 8

노드 : 3, 실행시간 :  0.00, 가중치합 : 15

노드 : 4, 실행시간 :  0.00, 가중치합 : 22

노드 : 5, 실행시간 :  0.00, 가중치합 : 27

노드 : 6, 실행시간 :  0.00, 가중치합 : 27

노드 : 7, 실행시간 :  0.00, 가중치합 : 34

노드 : 8, 실행시간 :  0.00, 가중치합 : 40

노드 : 9, 실행시간 :  0.00, 가중치합 : 56

노드 : 10, 실행시간 :  0.00, 가중치합 : 59

노드 : 11, 실행시간 :  0.01, 가중치합 : 88

노드 : 12, 실행시간 :  0.03, 가중치합 : 99

노드 : 13, 실행시간 :  0.05, 가중치합 : 132

노드 : 14, 실행시간 :  0.13, 가중치합 : 141

노드 : 15, 실행시간 :  0.31, 가중치합 : 191

노드 : 16, 실행시간 :  0.73, 가중치합 : 206

노드 : 17, 실행시간 :  1.67, 가중치합 : 262

노드 : 18, 실행시간 :  4.27, 가중치합 : 296

노드 : 19, 실행시간 :  9.62, 가중치합 : 362

노드 : 20, 실행시간 :  22.63, 가중치합 : 371

=== 테스트 결과 요약 ===

최대 처리 가능 노드 수 (60초 이내): 20

노드 수별 실행 시간:
2 노드 : 0.00 초
3 노드 : 0.00 초
4 노드 : 0.00 초
5 노드 : 0.00 초
6 노드 : 0.00 초
7 노드 : 0.00 초
8 노드 : 0.00 초
9 노드 : 0.00 초
10 노드 : 0.00 초
11 노드 : 0.01 초
12 노드 : 0.03 초
13 노드 : 0.05 초
14 노드 : 0.13 초
15 노드 : 0.31 초
16 노드 : 0.73 초
17 노드 : 1.67 초
18 노드 : 4.27 초
19 노드 : 9.62 초
20